## Preambule

In [5]:
# General packages
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path
import xarray as xr
import pandas as pd
import json
import plotly

# Plotting
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.colors import n_colors

## Parameters

In [6]:
rules = ['PCC', 'ECPC', 'AP']
rulecolors = ['goldenrod', 'tomato', 'forestgreen']

## Paths

In [7]:
path_main = Path("K:/Code/EffortSharing/")
path_data = Path("K:/Data/Data_effortsharing/DataUpdate_ongoing/")
path_figs = path_main / "Figures" / "Paper_FairShares"

## Read data files

In [8]:
xr_dataread = xr.open_dataset(path_data / "xr_dataread.nc")
xr_traj_2030 = xr.open_dataset(path_data / "xr_alloc_2030.nc")
xr_traj_2040 = xr.open_dataset(path_data / "xr_alloc_2040.nc")
xr_total = xr.open_dataset(path_data / "xr_policyscen.nc")

all_regions_iso = np.load(path_data / "all_regions.npy")
all_regions_names = np.load(path_data / "all_regions_names.npy")
all_countries_iso = np.load(path_data / "all_countries.npy", allow_pickle=True)
all_countries_names = np.load(path_data / "all_countries_names.npy", allow_pickle=True)

## Get Cost-optimal output

In [9]:
df_ar6 = pd.read_csv("X:/user/dekkerm/Data/IPCC/AR6_ISO3/AR6_Scenarios_Database_ISO3_v1.1.csv")
df_ar6 = df_ar6[df_ar6.Variable.isin(['Emissions|Kyoto Gases', 'Policy Cost|Consumption Loss'])]
df_ar6 = df_ar6.reset_index(drop=True)

df_ar6_meta = pd.read_excel("X:/user/dekkerm/Data/IPCC/AR6_ISO3/AR6_Scenarios_Database_metadata_indicators_v1.1.xlsx", sheet_name='meta_Ch3vetted_withclimate')
mods = np.array(df_ar6_meta.Model)
scens = np.array(df_ar6_meta.Scenario)
modscens_meta = np.array([mods[i]+'|'+scens[i] for i in range(len(scens))])
df_ar6_meta['ModelScenario'] = modscens_meta
df_ar6_meta = df_ar6_meta[['ModelScenario', 'Category', 'Policy_category']]
df_ar6_meta = df_ar6_meta[df_ar6_meta.Category.isin(['C1', 'C2'])]
df_ar6_meta = df_ar6_meta.reset_index(drop=True)
ms_meta_refined = np.array(df_ar6_meta.ModelScenario)

mods = np.array(df_ar6.Model)
scens = np.array(df_ar6.Scenario)
modscens = np.array([mods[i]+'|'+scens[i] for i in range(len(scens))])
df_ar6['ModelScenario'] = modscens
df_ar6 = df_ar6.drop(['Model', 'Scenario', 'Unit'], axis=1)
df_ar6 = df_ar6[df_ar6.ModelScenario.isin(np.array(ms_meta_refined))]

dummy = df_ar6.melt(id_vars=["Variable", "Region", "ModelScenario"], var_name="Time", value_name="Value")
dummy['Time'] = np.array(dummy['Time'].astype(int))
dummy = dummy.set_index(["Variable", "Region", "ModelScenario", "Time"])
xr_scen_r = xr.Dataset.from_dataframe(dummy)
xr_scen_r = xr_scen_r.reindex(Time = np.arange(1850, 2101))
xr_scen_r = xr_scen_r.interpolate_na(dim="Time", method="linear")

In [10]:
cost_opt = xr_scen_r.sel(Variable='Emissions|Kyoto Gases', Time=2040)/xr_scen_r.sel(Variable='Emissions|Kyoto Gases', Time=2040)*xr_scen_r.sel(Variable='Emissions|Kyoto Gases', Time=2040)

## Compute fractions

In [11]:
xr_larger_than_costopt = xr_traj_2040.sel(Temperature=1.5, Convergence_year=2050, Risk=0.5, NegEmis=0.5, NonCO2red=0.5, Timing='Immediate').median(dim=['Scenario']) > cost_opt.mean(dim='ModelScenario').Value

World

In [12]:
df_ar6 = pd.read_csv("X:/user/dekkerm/Data/IPCC/AR6_Scenarios_Database_World_v1.1.csv")
df_ar6 = df_ar6[df_ar6.Variable.isin(['Emissions|Kyoto Gases', 'Policy Cost|Consumption Loss'])]
df_ar6 = df_ar6.reset_index(drop=True)

df_ar6_meta = pd.read_excel("X:/user/dekkerm/Data/IPCC/AR6_Scenarios_Database_metadata_indicators_v1.1.xlsx", sheet_name='meta_Ch3vetted_withclimate')
mods = np.array(df_ar6_meta.Model)
scens = np.array(df_ar6_meta.Scenario)
modscens_meta = np.array([mods[i]+'|'+scens[i] for i in range(len(scens))])
df_ar6_meta['ModelScenario'] = modscens_meta
df_ar6_meta = df_ar6_meta[['ModelScenario', 'Category', 'Policy_category']]
df_ar6_meta = df_ar6_meta[df_ar6_meta.Category.isin(['C1', 'C2'])]
df_ar6_meta = df_ar6_meta.reset_index(drop=True)
ms_meta_refined = np.array(df_ar6_meta.ModelScenario)

mods = np.array(df_ar6.Model)
scens = np.array(df_ar6.Scenario)
modscens = np.array([mods[i]+'|'+scens[i] for i in range(len(scens))])
df_ar6['ModelScenario'] = modscens
df_ar6['Region'] = 'WORLD'
df_ar6 = df_ar6.drop(['Model', 'Scenario', 'Unit'], axis=1)
df_ar6 = df_ar6[df_ar6.ModelScenario.isin(np.array(ms_meta_refined))]

dummy = df_ar6.melt(id_vars=["Variable", "Region", "ModelScenario"], var_name="Time", value_name="Value")
dummy['Time'] = np.array(dummy['Time'].astype(int))
dummy = dummy.set_index(["Variable", "Region", "ModelScenario", "Time"])
xr_scen = xr.Dataset.from_dataframe(dummy)
xr_scen = xr_scen.reindex(Time = np.arange(1850, 2101))
xr_scen = xr_scen.interpolate_na(dim="Time", method="linear")

Regional

In [13]:
df_ar6 = pd.read_csv("X:/user/dekkerm/Data/IPCC/AR6_ISO3/AR6_Scenarios_Database_ISO3_v1.1.csv")
df_ar6 = df_ar6[df_ar6.Variable.isin(['Emissions|Kyoto Gases', 'Policy Cost|Consumption Loss'])]
df_ar6 = df_ar6.reset_index(drop=True)

df_ar6_meta = pd.read_excel("X:/user/dekkerm/Data/IPCC/AR6_ISO3/AR6_Scenarios_Database_metadata_indicators_v1.1.xlsx", sheet_name='meta_Ch3vetted_withclimate')
mods = np.array(df_ar6_meta.Model)
scens = np.array(df_ar6_meta.Scenario)
modscens_meta = np.array([mods[i]+'|'+scens[i] for i in range(len(scens))])
df_ar6_meta['ModelScenario'] = modscens_meta
df_ar6_meta = df_ar6_meta[['ModelScenario', 'Category', 'Policy_category']]
df_ar6_meta = df_ar6_meta[df_ar6_meta.Category.isin(['C1', 'C2'])]
df_ar6_meta = df_ar6_meta.reset_index(drop=True)
ms_meta_refined = np.array(df_ar6_meta.ModelScenario)

mods = np.array(df_ar6.Model)
scens = np.array(df_ar6.Scenario)
modscens = np.array([mods[i]+'|'+scens[i] for i in range(len(scens))])
df_ar6['ModelScenario'] = modscens
df_ar6 = df_ar6.drop(['Model', 'Scenario', 'Unit'], axis=1)
df_ar6 = df_ar6[df_ar6.ModelScenario.isin(np.array(ms_meta_refined))]

dummy = df_ar6.melt(id_vars=["Variable", "Region", "ModelScenario"], var_name="Time", value_name="Value")
dummy['Time'] = np.array(dummy['Time'].astype(int))
dummy = dummy.set_index(["Variable", "Region", "ModelScenario", "Time"])
xr_scen_r = xr.Dataset.from_dataframe(dummy)
xr_scen_r = xr_scen_r.reindex(Time = np.arange(1850, 2101))
xr_scen_r = xr_scen_r.interpolate_na(dim="Time", method="linear")

Concatenate

In [14]:
xr_scen_all = xr.merge([xr_scen, xr_scen_r])

## Data for plots

In [143]:
settings_default = {'Temperature': 1.6,
                   'Risk': 0.5,
                   'NegEmis': 0.5,
                   'NonCO2red': 0.5,
                   'Timing': "Immediate",
                   'Convergence_year': 2050,
                   'Scenario': 'SSP2',
                   'Discount_factor': 0,
                   'Historical_startyear': 1950,}
                   #'Capability_threshold': "Th",
                   #'RCI_weight': "Half"}
settings_ranges = {'Temperature': [1.6, 2.0],
                   'Risk': [0.5, 0.33],
                   'Convergence_year': [2040, 2050, 2060],
                    'NonCO2red':[0.33, 0.5, 0.67],
                   'Timing': ["Immediate"],
                   }
                   #'Capability_threshold': "Th",
                   #'RCI_weight': "Half"}

In [144]:
ndcs = xr_dataread.GHG_ndc.mean(dim=['Conditionality', 'Ambition'])
data = xr_traj_2030.sel(**settings_ranges)[['ECPC', 'AP', 'PCC']]
data_max = xr.concat([data.ECPC, data.AP, data.PCC], dim='variable2').max(dim=list(settings_default.keys())+['variable2'])
data = xr_traj_2030.sel(**settings_default)[['ECPC', 'AP', 'PCC']]
data_def = xr.concat([data.ECPC, data.AP, data.PCC], dim='variable2').max(dim='variable2')

In [146]:
print("Preferences on default settings")
print('-----')
data = xr_traj_2030.sel(**settings_default)[['ECPC', 'AP', 'PCC']]
data_def = xr.concat([data.ECPC, data.AP, data.PCC], dim='variable2').max(dim='variable2')
argmax_default = xr.concat([data.ECPC, data.AP, data.PCC], dim='variable2').fillna(-9e99).argmax(dim='variable2')

data = xr_traj_2030.sel(**settings_ranges)[['ECPC', 'AP', 'PCC']]
data_max = xr.concat([data.ECPC, data.AP, data.PCC], dim='variable2').max(dim=list(settings_default.keys())+['variable2'])
argmax_maximum = xr.concat([data.ECPC, data.AP, data.PCC], dim='variable2').fillna(-9e99).argmax(dim=list(settings_default.keys())+['variable2'])
x = xr.concat([data.ECPC, data.AP, data.PCC], dim='variable2').fillna(-9e99)
argmax_max = []
maxs = x.max(dim=list(settings_default.keys())+['variable2'])
for cty in x.Region:
    mx = float(maxs.sel(Region=cty).values)
    wh = np.where(x.sel(Region=cty).values == mx)
    wh = np.array(wh)[:, 0]
    argmax_max.append(wh[0])
argmax_max = np.array(argmax_max)

df_c = []
df_r_d = []
df_r_m = []
for cty_i, cty in enumerate(all_countries_iso):
    df_c.append(all_countries_names[cty_i])
    df_r_d.append(['ECPC', 'AP', 'PCC'][argmax_default.sel(Region=cty).values])
    df_r_m.append(['ECPC', 'AP', 'PCC'][argmax_max[cty_i]])
df_dict = {'Region': df_c, 'Default': df_r_d, 'Maximum': df_r_m}
df = pd.DataFrame(df_dict)
df.to_csv(path_data / "preferences_default_max.csv")

Preferences on default settings
-----


In [17]:
xr_policyscen = xr.open_dataset(path_data / "xr_policyscen.nc")
curpols = xr_policyscen.CurPol.sel(Time=2030).mean(dim='Model')
nzs = xr_policyscen.NetZero.sel(Time=2030).mean(dim='Model')

In [18]:
path_ctygroups = "X:/user/dekkerm/Data/" + "UNFCCC_Parties_Groups_noeu.xlsx"
df = pd.read_excel(path_ctygroups, sheet_name = "Country groups")
countries_iso = np.array(df["Country ISO Code"])
countries_name = np.array(df["Name"])
countries_iso = np.array(df["Country ISO Code"])
group_eu = countries_iso[np.array(df["EU"]) == 1]

In [19]:
costopts = xr_scen_all.sel(Time=2030, Variable='Emissions|Kyoto Gases').mean(dim='ModelScenario')

## New plot

In [183]:
fig = make_subplots(rows=2, cols=2,
                    specs=[[{'type': 'choropleth'}, {'type': 'choropleth'}],
                           [{'type': 'choropleth'}, {'type': 'choropleth'}]],
                    horizontal_spacing = 0., vertical_spacing=0.0)
for i in range(4):
    dath = [(ndcs-data_def)/data_def,
            (ndcs-data_max)/data_max,
            (costopts.Value)/data_def-1,
            (costopts.Value)/data_max-1][i]
    dat = np.array(dath)
    # if i>2:
    #     dat[dat < 0] = 0
    #scale = ['RdBu_r', 'RdBu_r', 'PuOr_r', 'PuOr_r'][i]
    scale = ['RdYlBu_r', 'RdYlBu_r', 'Earth_r', 'Earth_r'][i]
    dc = [dict(orientation='h', x=0.25, y=0.52, xanchor='left', yanchor='bottom', title='NDC deviation from emission allocation in 2030', tickvals=[-1, -0.5, 0, 0.5, 1],
                        ticktext=['100%<br>NDC sufficient <', '50%', '0%', '50%', '100%<br>> NDC insufficient'],
                        tickfont=dict(size=14),
                        len=0.5, lenmode='fraction', thickness=20, thicknessmode='pixels', titlefont=dict(size=14)),
          dict(orientation='h', x=0.25, y=0.52, xanchor='left', yanchor='bottom', title='NDC deviation from emission allocation in 2030', tickvals=[-1, -0.5, 0, 0.5, 1],
                        ticktext=['100%<br>NDC sufficient <', '50%', '0%', '50%', '100%<br>> NDC insufficient'],
                        tickfont=dict(size=14),
                        len=0.5, lenmode='fraction', thickness=20, thicknessmode='pixels', titlefont=dict(size=14)),
        dict(orientation='h', x=0.25, y=0.02, xanchor='left', yanchor='bottom', title='Fraction of mitigation in 2030 non-domestic', tickvals=[-1, -0.5, 0, 0.5, 1],
                        ticktext=['100%<br>More allocated than cost-optimal', '50%', '0%', '50%', '100%<br>Less allocated than cost-optimal'],
                        tickfont=dict(size=14),
                        len=0.5, lenmode='fraction', thickness=20, thicknessmode='pixels', titlefont=dict(size=14)),
        dict(orientation='h', x=0.25, y=0.02, xanchor='left', yanchor='bottom', title='Fraction of mitigation in 2030 non-domestic', tickvals=[-1, -0.5, 0, 0.5, 1],
                        ticktext=['100%<br>More allocated than cost-optimal', '50%', '0%', '50%', '100%<br>Less allocated than cost-optimal'],
                        tickfont=dict(size=14),
                        len=0.5, lenmode='fraction', thickness=20, thicknessmode='pixels', titlefont=dict(size=14))][i]
    
    # Grey out nans
    if i < 2:
        wh = np.where(np.isnan(dat))[0]
        fig.add_trace(go.Choropleth(
                locations=list(np.array(dath.Region)[wh])+['GRL'],
                z = [-0.5]*(len(dat[wh])+1),
                locationmode = 'ISO-3',
                colorscale ='Greys',
                zmax = 1,
                zmin = -1,
                hovertemplate  = '%{text}',
                name="",
                marker_line_color='black', 
                marker_line_width=0,
                showscale=False,
            ), [1, 1, 2, 2][i], [1, 2, 1, 2][i])
    else:
        fig.add_trace(go.Choropleth(
                locations=list(np.array(data_def.Region))+['GRL'],
                z = [-0.5]*(len(np.array(data_def.Region))+1),
                locationmode = 'ISO-3',
                colorscale ='Greys',
                zmax = 1,
                zmin = -1,
                hovertemplate  = '%{text}',
                name="",
                marker_line_color='black', 
                marker_line_width=0,
                showscale=False,
            ), [1, 1, 2, 2][i], [1, 2, 1, 2][i])
    
    fig.add_trace(go.Choropleth(
            locations=np.array((dath).Region),
            z = dat,
            locationmode = 'ISO-3',
            colorscale = scale,
            zmax = 1,
            zmin = -1,
            text = [str(r)+": "+str(np.round(100*dat[r_i], 1))+" %" for r_i, r in enumerate(np.array((dath).Region))],
            hovertemplate  = '%{text}',
            name="",
            colorbar=dc,
            marker_line_color='black', 
            marker_line_width=0.1,
            showscale=True,
        ), [1, 1, 2, 2][i], [1, 2, 1, 2][i])
    
    # EU
    fig.add_trace(go.Choropleth(
            locations=group_eu,
            z = [dat[np.where(np.array((dath).Region) == 'EU')[0][0]]]*len(group_eu),
            locationmode = 'ISO-3',
            colorscale =scale,
            zmax = 1,
            zmin = -1,
            #text = [str(r)+": "+str(np.round(100*dat[r_i], 1))+" %" for r_i, r in enumerate(group_eu)],
            hovertemplate  = '%{text}',
            name="",
            marker_line_color='black', 
            marker_line_width=0.1,
            showscale=False,
        ), [1, 1, 2, 2][i], [1, 2, 1, 2][i])
    
    # Emphasize >0
    wh = np.where(np.array(dath) > 0)[0]
    fig.add_trace(go.Choropleth(
            locations=np.array((dath).Region)[wh],
            z = dat[wh],
            locationmode = 'ISO-3',
            colorscale =scale,
            zmax = 1,
            zmin = -1,
            text = [str(r)+": "+str(np.round(100*dat[wh][r_i], 1))+" %" for r_i, r in enumerate(np.array((dath).Region)[wh])],
            hovertemplate  = '%{text}',
            name="",
            marker_line_color='black', 
            marker_line_width=2,
            showscale=False,
        ), [1, 1, 2, 2][i], [1, 2, 1, 2][i])
    
    if 'EU' in np.array((dath).Region)[wh]:
        fig.add_trace(go.Choropleth(
                locations=group_eu,
                z = [dat[np.where(np.array((dath).Region) == 'EU')[0][0]]]*len(group_eu),
                locationmode = 'ISO-3',
                colorscale =scale,
                zmax = 1,
                zmin = -1,
                #text = [str(r)+": "+str(np.round(100*dat[r_i], 1))+" %" for r_i, r in enumerate(group_eu)],
                hovertemplate  = '%{text}',
                name="",
                marker_line_color='black', 
                marker_line_width=2,
                showscale=False,
        ), [1, 1, 2, 2][i], [1, 2, 1, 2][i])

fs=20
fig['layout'].update(
    annotations=[dict(x=0.01, y=1.02, text='<b>(a) Evaluation NDCs in light of default allocations</b>', xref='paper', yref='paper', showarrow=False, font=dict(color='black', size=fs)),
                 dict(x=0.895, y=1.02, text='<b>(b) Evaluation NDCs in light of maximum allocations', xref='paper', yref='paper', xanchor='right', showarrow=False, font=dict(color='black', size=fs)),
                 dict(x=0.01, y=0.5, text='<b>(c) Comparison cost-optimal pathways to default allocations', xref='paper', yref='paper', xanchor='left', showarrow=False, font=dict(color='black', size=fs)),
                 dict(x=0.98, y=0.5, text='<b>(d) Comparison cost-optimal pathways to maximum allocations', xref='paper', yref='paper', xanchor='right', showarrow=False, font=dict(color='black', size=fs)),

                 dict(x=0.0, y=0.12, text='<b>United States</b> has 53% more<br>cost-optimal emissions (in 2030)<br>than would be fair to emit<br>on default settings', xref='paper', yref='paper', xanchor='left', showarrow=False, font=dict(color='black', size=14)),
                 dict(x=0.99, y=0.04, text='When looking at the highest potentially<br>fair allocation for <b>Australia</b>,<br>this country is allocated equal emissions<br>as would be cost-optimal to emit here', xref='paper', yref='paper', xanchor='right', showarrow=False, font=dict(color='black', size=14)),
                 dict(x=0.83, y=0.25, text='<b>Russia</b> is always allocated less<br>than is cost-optimal to emit there', xref='paper', yref='paper', xanchor='right', showarrow=False, font=dict(color='black', size=14)),
                 dict(x=0.95, y=0.59, text='All reddish/bold countries here have<br>insufficient NDCs, even when looking<br>at the highest potentially fair allocation', xref='paper', yref='paper', xanchor='right', showarrow=False, font=dict(color='black', size=14)),
                 dict(x=0.0, y=0.59, text='Most of the Global North together<br>with countries like Australia and Argentina<br>have insufficient NDCs when compared<br>to default fair share allocations', xref='paper', yref='paper', xanchor='left', showarrow=False, font=dict(color='black', size=14)),
                 ])

fig.update_geos(visible=False,
                showlakes=False,
                lakecolor='rgb(255, 255, 255)',
                projection_type='natural earth',
                resolution=110,
                showcoastlines=False,)
fig.update_traces(colorbar=dict(
    title=dict(
        side="bottom"
    )
))
fig.update_geos(landcolor="grey")
args = dict(xref="paper", yref="paper", type='line')
fig.add_shape(x0=0.08, y0=0.20, x1=0.12, y1=0.35, **args, line=dict(color='black', width=2))
fig.add_shape(x0=0.75, y0=0.29, x1=0.85, y1=0.40, **args, line=dict(color='black', width=2))
fig.add_shape(x0=0.90, y0=0.12, x1=0.93, y1=0.18, **args, line=dict(color='black', width=2))

fig.add_shape(x0=0.08, y0=0.63, x1=0.12, y1=0.85, **args, line=dict(color='black', width=2))
fig.add_shape(x0=0.87, y0=0.62, x1=0.85, y1=0.90, **args, line=dict(color='black', width=2))
fig.update_layout(height=1100, width=1700)
fig.write_image("K:/Code/EffortSharing/Figures/Paper_FairShares/Figure_4.png", scale=5)
fig.show()

## Get country list

In [153]:
for i in range(4):
    dath = [(ndcs-data_def)/data_def,
            (ndcs-data_max)/data_max,
            (costopts.Value)/data_def-1,
            (costopts.Value)/data_max-1][i]
    wh = np.where(np.array(dath) > 0)[0]
    print(np.array((dath).Region)[wh])

['ALB' 'DZA' 'AND' 'ARG' 'AUS' 'AZE' 'BRB' 'BLR' 'BIH' 'BWA' 'BDI' 'KHM'
 'CAN' 'CAF' 'CHN' 'COL' 'DMA' 'EGY' 'GNQ' 'GEO' 'ISL' 'IDN' 'IRN' 'ISR'
 'JAM' 'JPN' 'KAZ' 'KWT' 'LBN' 'LIE' 'MKD' 'MYS' 'MDV' 'MHL' 'MUS' 'MEX'
 'MDA' 'MNG' 'NZL' 'PRK' 'OMN' 'PRY' 'PER' 'RUS' 'KNA' 'LCA' 'SAU' 'SYC'
 'SGP' 'SLB' 'ZAF' 'KOR' 'THA' 'BHS' 'TTO' 'TUR' 'UKR' 'ARE' 'GBR' 'USA'
 'UZB' 'VEN' 'VNM' 'EU' 'EARTH']
['AND' 'BWA' 'CHN' 'GEO' 'ISL' 'IRN' 'JAM' 'KWT' 'LIE' 'MYS' 'MHL' 'PRK'
 'PER' 'RUS' 'KNA' 'SAU' 'SGP' 'BHS' 'TTO' 'TUR' 'UZB' 'VEN' 'EARTH']
['ARG' 'AUS' 'CAN' 'CHN' 'COL' 'EU' 'JPN' 'KOR' 'MEX' 'RUS' 'SAU' 'TUR'
 'USA' 'ZAF']
['ARG' 'RUS' 'SAU']


## Plot

In [21]:
# path_ctygroups = "X:/user/dekkerm/Data/" + "UNFCCC_Parties_Groups_noeu.xlsx"
# df = pd.read_excel(path_ctygroups, sheet_name = "Country groups")
# countries_iso = np.array(df["Country ISO Code"])
# countries_name = np.array(df["Name"])
# countries_iso = np.array(df["Country ISO Code"])
# group_eu = countries_iso[np.array(df["EU"]) == 1]

Helper stuff

In [22]:
# colmap = 'Earth_r'
# colmap2 = 'terrain_r'

In [23]:
# regs = xr_larger_than_costopt.Region#np.array(larger_than_costopt.Region)[np.where(~np.isnan(larger_than_costopt))]
# data = []
# for cty_i, cty in enumerate(regs):
#     data.append(np.sum([float(xr_larger_than_costopt.sel(Region=cty)[i]) for i in rules]))
#     #larger_than_costopt.sel(Region=regs)/6
# data = (6-np.array(data))/6

Actual plot

In [24]:
# fig = make_subplots(rows=5, cols=4, 
#                     specs=[[{'type': 'choropleth', 'colspan': 3, 'rowspan': 3}, {}, {}, {}],
#                            [{}, {}, {}, {}],
#                            [{}, {}, {}, {}],
#                            [{}, {}, {}, {}],
#                            [{'colspan': 4}, {}, {}, {}]],
#                     horizontal_spacing = 0.05, vertical_spacing=0.08)

# fig.add_trace(
#     go.Choropleth(
#     locations=regs,
#     z = data,
#     locationmode = 'ISO-3',
#     colorscale = colmap,
#     zmax = 1,
#     zmin = 0,
#     text = [str(r)+": "+str(int(data[r_i]*6))+" out of 6" for r_i, r in enumerate(np.array(regs))],
#     hovertemplate  = '%{text}',
#     name="",
#     marker_line_color='black', 
#     marker_line_width=1.,
#     colorbar=dict(len=0.43, x=0.37, y=0.45, orientation='h',
#                   tickvals = np.arange(0, 1.1, 1/6),
#                   thickness=15,
#                   tickfont={'size': 10},
#                   ticktext=['None<br>(Mitigation financed<br>by others)', '1 out of 6', '2 out of 6', '3 out of 6', '4 out of 6', '5 out of 6', 'All<br>(Finances mitigation<br>abroad)'],
#                   ticklabelposition='outside',
#     ),
#     showscale=True
# ), 1, 1)
# fig.add_trace(
#     go.Choropleth(
#     locations=group_eu,
#     z = [data[np.where(regs == "EU")[0][0]]]*len(group_eu),
#     locationmode = 'ISO-3',
#     colorscale = colmap,
#     zmax = 1,
#     zmin = 0,
#     text = ["EU: "+str(int(6*data[regs == 'EU']))+" out of 6"for r_i, r in enumerate(group_eu)],
#     hovertemplate  = '%{text}',
#     name="",
#     marker_line_color='black', 
#     marker_line_width=1.,
#     showscale=False
# ), 1, 1)

# # limite map scope to USA 'africa', 'asia', 'europe', 'north america', 'south america', 'usa', 'world'
# fig.update_geos(showocean=True, oceancolor="aliceblue")
# fig.update_geos(showlakes=True, lakecolor="aliceblue")
# fig.update_geos(visible=True,
#                 showlakes=False,
#                 lakecolor='rgb(255, 255, 255)',
#                 projection_type='natural earth',
#                 showcoastlines=False,)

# values = ['CP', 'NDC', 'NZ', 'CO2_costopt', 'GF', 'PC', 'PCC', 'AP', 'GDR', 'ECPC']
# for cty_i, cty in enumerate(["USA", "BRA", "ZAF", "EU", "RUS", "CHN", "IND"]):
#     row = [4, 4, 4, 1, 2, 3, 4][cty_i]
#     col = [1, 2, 3, 4, 4, 4, 4][cty_i]
#     # !!!!! values_co_emis = xr_scen_all.sel(Region=cty, Time=np.arange(params['start_year_analysis'], 2101), Variable='Emissions|Kyoto Gases').Value.sum(dim='Time', skipna=False)
#     values_co_emis = xr_scen_all.sel(Region=cty, Time=2040, Variable='Emissions|Kyoto Gases').Value/1e3
#     fig.add_trace(go.Scatter(x=['CP', 'NDC', 'NZ', 'CO', 'GF', 'PC', 'PCC', 'AP', 'GDR', 'ECPC'], y=[values_co_emis.median() for val in values], name='', showlegend=False, hoverinfo='skip', mode='lines', line={'color': 'black', 'width': 0}), col=col, row=row)
#     fig.add_trace(go.Scatter(x=['CO', 'GF', 'PC', 'PCC', 'AP', 'GDR', 'ECPC'], y=[values_co_emis.median() for val in values], name='', showlegend=False, hoverinfo='skip', mode='lines', line={'color': 'black', 'width': 1}), col=col, row=row)
#     fig.add_trace(go.Scatter(x=['GF', 'PC', 'PCC', 'AP', 'GDR', 'ECPC'], y=[values_co_emis.quantile(0.25) for val in values], fill=None, name='', showlegend=False, hoverinfo='skip', mode='lines', line={'color': 'silver', 'width': 0}), col=col, row=row)
#     fig.add_trace(go.Scatter(x=['GF', 'PC', 'PCC', 'AP', 'GDR', 'ECPC'], y=[values_co_emis.quantile(0.75) for val in values], fill='tonexty', name='', showlegend=False, hoverinfo='skip', mode='lines', line={'color': 'silver', 'width': 0}), col=col, row=row)
#     fig.add_trace(go.Box(y=values_co_emis,
#                         name="CO",
#                         #text=[cty+": "+str(np.round(vals[i],1))+"<br>"+modscens[i] for i in range(len(vals))],
#                         fillcolor=plotly.colors.diverging.Earth_r[int(data[regs == cty][0]*6)],
#                         line_color='black', marker=dict(color='grey'),#plotly.colors.diverging.Earth_r[int(fractions[regs == cty][0]*7)]),
#                         boxpoints='all', jitter=0.5, pointpos=0, whiskerwidth=0.4, marker_size=0.1, line_width=1.5, showlegend=False, hovertemplate='%{text}'),
#                         row=row, col=col)

#     # Engage results
#     for eng_i, eng in enumerate(['CurPol', 'NDC', 'NetZero']):
#         # !!!!! values_eng = xr_total.sel(Time=np.arange(params['start_year_analysis'], 2101), Region=cty).sum(dim='Time', skipna=False)[eng]
#         values_eng = xr_total.sel(Time=2040, Region=cty)[eng]/1e3
#         fig.add_trace(go.Box(y=values_eng,
#                         name=["CP", "NDC", "NZ"][eng_i],
#                         fillcolor=plotly.colors.diverging.Earth_r[int(data[regs == cty][0]*6)],
#                         line_color='black', marker=dict(color='grey'),
#                         boxpoints='all', jitter=0.5, pointpos=0, whiskerwidth=0.4, marker_size=0.1, line_width=1.5, showlegend=False, hovertemplate='%{text}'),
#                         row=row, col=col)

#     # Full cost optimal emissions (AR6 C1-C2)
#     fig.add_trace(go.Box(y=values_co_emis,
#                     name="CO",
#                     fillcolor=plotly.colors.diverging.Earth_r[int(data[regs == cty][0]*6)],
#                     line_color='black', marker=dict(color='grey'),
#                     boxpoints='all', jitter=0., pointpos=0, whiskerwidth=0.4, marker_size=0.1, line_width=1.5, showlegend=False, hovertemplate='%{text}'),
#                     row=row, col=col)

#     # Effort sharing
#     for alloc_i, alloc in enumerate(['GF', 'PC', 'PCC', 'AP', 'GDR', 'ECPC']):
#         # !!!!! ally = xr_bud15.sel(Region=cty, Risk=0.5, Convergence_year=2050).sum(dim='Time').mean(dim='Scenario')[alloc]
#         ally = xr_traj_2040.sel(Region=cty, Risk=0.5, Convergence_year=2050, Temperature=1.5).mean(dim='Scenario')[alloc].median()/1e3
#         if ally < values_co_emis.quantile(0.40):
#             colr = 'tomato'
#             marker = 'triangle-down'
#             textpos = 'middle left'
#             txt = '-<b>$'
#             size = 13
#         elif ally > values_co_emis.quantile(0.60):
#             colr = 'forestgreen'
#             marker = 'triangle-up'
#             textpos = 'middle left'
#             txt = '+<b>$'
#             size = 13
#         else:
#             colr = 'grey'
#             marker = 'square'
#             textpos = 'middle left'
#             txt = '~<b>$'
#             size = 10
#         fig.add_trace(go.Scatter(x=[alloc, alloc], y=[values_co_emis.median(), ally], mode='lines', line={'width': 2, 'color': colr, 'dash': 'dot'}, showlegend=False), row=row, col=col)
#         fig.add_trace(go.Scatter(x=[alloc], y=[ally], text=txt, textfont={'color': colr}, mode='markers+text', marker={'symbol':marker, "color": colr, "size": size, "line": {"width": 1.5, "color": 'black'}},
#                                  showlegend=False, textposition=textpos),
#                       row=row, col=col)

# y1 = xr_scen_all.sel(Time=np.arange(params['start_year_analysis'], 2101), Variable='Policy Cost|Consumption Loss').Value.sum(dim='Time', skipna=False)
# gdp1 = xr_dataread.sel(Time=params['start_year_analysis']).GDP.mean(dim='Scenario')
# ar = np.array((y1/gdp1).Region[np.array((y1/gdp1).median(dim='ModelScenario').argsort())])
# ar = ar[ar!='WORLD']
# for reg_i, reg in enumerate(ar):
#     y = xr_scen_all.sel(Region=reg, Time=np.arange(params['start_year_analysis'], 2101), Variable='Policy Cost|Consumption Loss').Value.sum(dim='Time', skipna=False)
#     gdp = xr_dataread.sel(Region=reg, Time=params['start_year_analysis']).GDP.mean(dim='Scenario')
#     try:
#         col = plotly.colors.diverging.Earth_r[int(data[regs == reg][0]*6)]
#     except:
#         col = 'silver'
#     if np.nansum(y) != 0:
#         fig.add_trace(go.Box(name=reg, y=y/gdp,
#                             line_color='black',
#                             marker=dict(color=col),
#                             fillcolor=col,
#                             boxpoints='all', jitter=0.5, pointpos=-1.8, whiskerwidth=0.4, marker_size=3, line_width=1,
#                             showlegend=False), row=5, col=1)

# # Panel labels
# fig.add_annotation(xref='paper', yref='paper',ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=18),
#                    y=1.04, x=0.20, text="<b>(a) Allocations less than cost-optimal</b><br>          (in number of effort-sharing approaches)")
# fig.add_annotation(xref='paper', yref='paper',ax=0, align='right', ay=-0, showarrow=False, font=dict(color='black', size=16), y=0.38, x=0.01, text="<b>(b) United States of America")
# fig.add_annotation(xref='paper', yref='paper',ax=0, align='right', ay=-0, showarrow=False, font=dict(color='black', size=16), y=0.38, x=0.27, text="<b>(c) Brazil")
# fig.add_annotation(xref='paper', yref='paper',ax=0, align='right', ay=-0, showarrow=False, font=dict(color='black', size=16), y=0.38, x=0.60, text="<b>(d) South-Africa")
# fig.add_annotation(xref='paper', yref='paper',ax=0, align='right', ay=-0, showarrow=False, font=dict(color='black', size=16), y=1.02, x=0.94, text="<b>(e) European Union")
# fig.add_annotation(xref='paper', yref='paper',ax=0, align='right', ay=-0, showarrow=False, font=dict(color='black', size=16), y=0.81, x=0.87, text="<b>(f) Russia")
# fig.add_annotation(xref='paper', yref='paper',ax=0, align='right', ay=-0, showarrow=False, font=dict(color='black', size=16), y=0.58, x=0.87, text="<b>(g) China")
# fig.add_annotation(xref='paper', yref='paper',ax=0, align='right', ay=-0, showarrow=False, font=dict(color='black', size=16), y=0.38, x=0.87, text="<b>(h) India")
# fig.add_annotation(xref='paper', yref='paper',ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=16), y=0.14, x=0.01, text="<b>(i) Policy costs over 2020-2100 in cost-optimal scenarios</b>")

# # ADD LINES TO PATHS
# fig.update_yaxes(title_text="GHG emission allocations in 2040<br>[Gt CO<sub>2</sub>e]", row=4, col=1)
# fig.update_yaxes(title_text="Cumulative consumption<br>loss as fraction<br>of GDP in 2019", row=5, col=1)

# fig.update_yaxes(rangemode="tozero")
# fig.update_xaxes(zeroline=True, zerolinewidth=2, zerolinecolor='Black')
# fig.update_layout(height=1100,
#                   width=1400,
#                   template='simple_white')
# fig.show()

In [25]:
# fig.write_image(path_figs / "Figure_4.svg")